In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/Shareddrives/AI_Project/

/content/drive/Shareddrives/AI_Project


In [3]:
import csv
path = 'sentence_train.csv'
sentence_word_tag = list(csv.reader(open(path, encoding="utf8")))[1:]
# print(sentence_word_tag)

In [4]:
transition_table = {}
emission_table = {}
transition_count = {}
emission_count = {}
all_tags = set()
all_words = set()

In [5]:
for sentence in sentence_word_tag:
  sentence = ' '.join(sentence)
  sentence = sentence.split(' ')
  sentence = sentence[1:-1]
  prev_tags = ["start"]
  for word_tag in sentence:
    try:
      word = word_tag.split('_')[0]
      curr_tags = word_tag.split('_')[1].split('-')
    except:
      continue
    for curr_tag in curr_tags:
      for prev_tag in prev_tags:
        if prev_tag not in transition_table:
          all_tags.add(prev_tag)
          transition_table[prev_tag] = {}
          transition_count[prev_tag] = 0
        if curr_tag not in transition_table[prev_tag]:
          all_tags.add(curr_tag)
          transition_table[prev_tag][curr_tag] = 0
        transition_table[prev_tag][curr_tag] = transition_table[prev_tag][curr_tag] + 1
        transition_count[prev_tag] = transition_count[prev_tag] + 1
        if (prev_tag[0]=="start"):
          continue
        if curr_tag not in emission_table:
          emission_table[curr_tag] = {}
          emission_count[curr_tag] = 0
        if word not in emission_table[curr_tag]:
          all_words.add(word)
          emission_table[curr_tag][word] = 0
        emission_table[curr_tag][word] = emission_table[curr_tag][word] + 1 
        emission_count[curr_tag] = emission_count[curr_tag] + 1
    prev_tags = curr_tags
  curr_tags = ["end"]
  for curr_tag in curr_tags:
    for prev_tag in prev_tags:
      if prev_tag not in transition_table:
        transition_table[prev_tag] = {}
        transition_count[prev_tag] = 0
      if curr_tag not in transition_table[prev_tag]:
        transition_table[prev_tag][curr_tag] = 0
      transition_table[prev_tag][curr_tag] = transition_table[prev_tag][curr_tag] + 1
      transition_count[prev_tag] = transition_count[prev_tag] + 1

In [6]:
for prev_tag in transition_table:
  for curr_tag in transition_table[prev_tag]:
    transition_table[prev_tag][curr_tag] = transition_table[prev_tag][curr_tag]/transition_count[prev_tag]
for tag in emission_table:
  for word in emission_table[tag]:
    emission_table[tag][word] = emission_table[tag][word]/emission_count[tag]

In [7]:
def HMM(sentence):
  sentence = ' '.join(sentence)
  sentence = sentence.split(' ')
  sentence = sentence[1:-1]
  prev_state = {}
  curr_state = {}
  prev_state['start'] = {}
  prev_state['start']['sequence'] = ['start']
  prev_state['start']['probability'] = 1
  for word_tag in sentence:
    try:
      word = word_tag.split('_')[0]
      acc_tags = word_tag.split('_')[1].split('-')
    except:
      continue
    for tag in all_tags:
      curr_state[tag] = {}
      curr_state[tag]['sequence'] = []
      curr_state[tag]['probability'] = 0
    for prev_tag in prev_state:
      for curr_tag in transition_table[prev_tag]:
        if word in all_words:
          if (curr_tag=='end'):
            continue
          if word in emission_table[curr_tag]:
            probability = prev_state[prev_tag]['probability']*transition_table[prev_tag][curr_tag]*emission_table[curr_tag][word]
            if (probability>curr_state[curr_tag]['probability']):
              curr_state[curr_tag]['probability'] = prev_state[prev_tag]['probability']*transition_table[prev_tag][curr_tag]*emission_table[curr_tag][word]
              curr_state[curr_tag]['sequence'] = prev_state[prev_tag]['sequence'][:]
              curr_state[curr_tag]['sequence'].append(curr_tag)
          else:
            if curr_tag in curr_state:
              del curr_state[curr_tag]  
        else:
          if (curr_tag=='end'):
            continue
          probability = prev_state[prev_tag]['probability']*transition_table[prev_tag][curr_tag]
          if (probability>curr_state[curr_tag]['probability']):
            curr_state[curr_tag]['probability'] = prev_state[prev_tag]['probability']*transition_table[prev_tag][curr_tag]
            curr_state[curr_tag]['sequence'] = prev_state[prev_tag]['sequence'][:]
            curr_state[curr_tag]['sequence'].append(curr_tag)
    prev_state = {}
    for tag in curr_state:
      if(curr_state[tag]['probability']!=0):
        prev_state[tag] = {}
        prev_state[tag]['sequence'] = curr_state[tag]['sequence'][:]
        prev_state[tag]['probability'] = curr_state[tag]['probability']
    curr_state = {}
  curr_state = {}
  curr_state['end'] = {}
  curr_state['end']['sequence'] = []
  curr_state['end']['probability'] = 0
  for prev_tag in prev_state:
    if 'end' in transition_table[prev_tag]:
      probability = prev_state[prev_tag]['probability']*transition_table[prev_tag]['end']
      if (probability>curr_state['end']['probability']):
        curr_state['end']['sequence'] = prev_state[prev_tag]['sequence'][:]
        curr_state['end']['sequence'].append('end')
        curr_state['end']['probability'] = probability

  return curr_state['end']['sequence']

In [8]:
path = 'sentence_test.csv'
sentence_word_tag = list(csv.reader(open(path, encoding="utf8")))[1:]

In [9]:
hit = 0 
miss = 0
lines = 0
for sentence in sentence_word_tag[1:]:
  lines = lines + 1
  seq = HMM(sentence)
  cnt = 1
  sentence = ' '.join(sentence)
  sentence = sentence.split(' ')
  sentence = sentence[1:-1]    
  for word_tag in sentence:
    try:
      word = word_tag.split('_')[0]
      acc_tags = word_tag.split('_')[1].split('-')
    except:
      continue
    flag = 0
    for tag in acc_tags:
      try:
        if (tag==seq[cnt]):
          flag = 1
      except:
        flag = 0
    if (flag):
      hit = hit + 1
    else :
      miss = miss + 1
    cnt = cnt + 1
  if(lines%5000==0):
    print("{} : hits = {}  , miss = {} , accuracy = {}".format(lines,hit,miss,hit/(hit+miss)))

5000 : hits = 101837  , miss = 2830 , accuracy = 0.9729618695481862
10000 : hits = 176674  , miss = 5561 , accuracy = 0.9694844568825967
15000 : hits = 244056  , miss = 8508 , accuracy = 0.9663134888582696
20000 : hits = 339297  , miss = 11534 , accuracy = 0.9671237718445633
25000 : hits = 437112  , miss = 14717 , accuracy = 0.9674279428721928
30000 : hits = 533597  , miss = 17303 , accuracy = 0.9685913958976221
35000 : hits = 645264  , miss = 21198 , accuracy = 0.9681932353232442
40000 : hits = 783410  , miss = 25442 , accuracy = 0.968545543560503
45000 : hits = 859352  , miss = 28932 , accuracy = 0.9674293356629187
50000 : hits = 962605  , miss = 32411 , accuracy = 0.9674266544457577
55000 : hits = 1062467  , miss = 35646 , accuracy = 0.9675388598441144
60000 : hits = 1190207  , miss = 39953 , accuracy = 0.9675221109449177
65000 : hits = 1290377  , miss = 43652 , accuracy = 0.9672780726655867
70000 : hits = 1363738  , miss = 46254 , accuracy = 0.9671955585563606
75000 : hits = 146140

In [10]:
print("Overall accuracy on Test Dataset")
print("hits = {}  , miss = {} , accuracy = {}".format(hit,miss,hit/(hit+miss)))

Overall accuracy on Test Dataset
hits = 3990290  , miss = 139451 , accuracy = 0.9662325070748989
